In [3]:
import pandas as pd
pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", 1000)
import numpy as np
import scipy as scp
import scipy.stats as ss
import sklearn.metrics as sm

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import quad
import matplotlib.pyplot as plt

import scipy.special as scps
from statsmodels.graphics.gofplots import qqplot
from scipy.linalg import cholesky
from functools import partial
from scipy.optimize import minimize, brute, fmin
from IPython.display import display
import sympy; sympy.init_printing()

import seaborn as sns
from tqdm import tqdm
init_notebook_mode(connected=True)

def display_matrix(m):
    display(sympy.Matrix(m))

## Import Data and Visualization

### Implied Volatility

In [4]:
impl_vol = pd.read_csv("./data/optionm_opprcd_SPX_1996-2017.csv")
impl_vol.set_index('date', drop=False, inplace=True)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
sp500 = pd.read_csv('./data/sp500index.csv')
sp500.Date = sp500.Date.apply(lambda x: x.replace('-','')).astype('int64', copy=False)
sp500.set_index('Date', drop=False, inplace=True)

In [6]:
impl_vol = impl_vol.join(sp500)

In [7]:
impl_vol.head()

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,cfadj,ss_flag,root,suffix,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag,Date,Open,High,Low,Close,Adj Close,Volume
19960104,108105,19960104,098A3.1A,0,19960316,19960104.0,C,600000,24.750,25.750,150,5633,0.110927,0.768930,0.010008,81.23895,-37.061410,10003226,1,0,098A3,1A,64881510,SPX,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,1,19960104.0,621.320007,624.48999,613.960022,617.700012,617.700012,512580000.0
19960104,108105,19960104,09A49.0B,0,19960316,19960104.0,P,615000,10.125,10.875,156,8302,0.125890,-0.412243,0.011389,104.91490,-25.811090,10111243,1,0,09A49,0B,64881510,SPX,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,1,19960104.0,621.320007,624.48999,613.960022,617.700012,617.700012,512580000.0
19960104,108105,19960104,098B6.4F,0,19970621,19960104.0,P,550000,11.625,13.250,5,3282,0.159369,-0.175387,0.002143,189.96390,-6.747066,10008143,1,0,098B6,4F,64881510,SPX,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,1,19960104.0,621.320007,624.48999,613.960022,617.700012,617.700012,512580000.0
19960104,108105,19960104,099C9.98,0,19960316,19960104.0,P,600000,6.000,6.500,3917,5728,0.136913,-0.267751,0.008869,88.85873,-26.149320,10078616,1,0,099C9,98,64881510,SPX,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,1,19960104.0,621.320007,624.48999,613.960022,617.700012,617.700012,512580000.0
19960104,108105,19960104,09C69.1E,0,19960316,19960104.0,P,560000,1.750,2.000,3,2971,0.176965,-0.083058,0.003191,41.32092,-17.340430,10250526,1,0,09C69,1E,64881510,SPX,9999,1,32768,NaN,A,NaN,CBOE S&P 500 INDEX,I,E,1,19960104.0,621.320007,624.48999,613.960022,617.700012,617.700012,512580000.0


In [8]:
def t2m(isdate, exdate, verbose = 0):
    isdate = pd.to_datetime(isdate, format='%Y%m%d')
    exdate = pd.to_datetime(exdate, format='%Y%m%d')
    return (exdate - isdate)
        
t2m('20170101', '20180101', 1)

Timedelta('365 days 00:00:00')

In [9]:
impl_vol['t2m'] = t2m(impl_vol.loc[:, 'date'], impl_vol.loc[:, 'exdate']).apply(lambda x: x.days/365)

In [10]:
date = impl_vol.date == 20170829
days = impl_vol.exdate >0
cp_flag = impl_vol.cp_flag=="C"
volume = impl_vol.volume > 0

toplot = date & \
         days & \
         cp_flag &\
         volume
 
impl_vol.loc[toplot, ['date','exdate','t2m','impl_volatility', 'volume', 'strike_price']]

,date,exdate,t2m,impl_volatility,volume,strike_price
20170829,20170829,20170915,0.046575,NaN,6,1300000
20170829,20170829,20170915,0.046575,NaN,3,1800000
20170829,20170829,20170915,0.046575,NaN,3,1880000
20170829,20170829,20170915,0.046575,NaN,1,1925000
20170829,20170829,20170915,0.046575,NaN,1,1975000
20170829,20170829,20170915,0.046575,NaN,1,1985000
20170829,20170829,20170915,0.046575,NaN,30,2000000
20170829,20170829,20170915,0.046575,0.229659,307,2150000
20170829,20170829,20170915,0.046575,0.223415,60,2175000
20170829,20170829,20170915,0.046575,0.216554,1,2200000


In [11]:
impl_vol.loc[20171228,['Adj Close']]

,Adj Close
20171228,2687.540039
20171228,2687.540039
20171228,2687.540039
20171228,2687.540039
20171228,2687.540039
...,...
20171228,2687.540039
20171228,2687.540039
20171228,2687.540039
20171228,2687.540039


In [12]:
str(round(t2m_mask[i] ,4))

NameError: name 't2m_mask' is not defined

In [23]:
date = impl_vol.date == 20170829
days = impl_vol.exdate >0#== 20170915
cp_flag = impl_vol.cp_flag=="C"
volume = impl_vol.volume > 0

toplot = date &\
         days &\
         cp_flag &\
         volume

data_toplot = impl_vol.loc[toplot,:]

data_toplot = data_toplot.sort_values(by=['strike_price'])

t2m_mask = np.unique(data_toplot.loc[:, 't2m'])

mask = [data_toplot.loc[:, 't2m'] == t2m_mask[i] for i in range(len(t2m_mask))]


data_toplot1 = [go.Scatter(visible = False,
                     name = str(round(t2m_mask[i] ,4)),
                     mode = 'markers+lines',
                     x = data_toplot.loc[m, 'strike_price']/1000, 
                     y = data_toplot.loc[m, 'impl_volatility'], 
                     )
                for i,m in enumerate(mask)]

steps = []
for i in range(len(data_toplot1)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data_toplot1)], 
        label=str(round(t2m_mask[i] ,4))
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 1,
    currentvalue = {"prefix": "Time to Maturity:"},
#     pad = {"t": 50},
    steps = steps
)]

strike_price_toplot = data_toplot.iloc[0,-2]/1000000
# ymin = np.min(data_toplot.loc[:, 'impl_volatility'])
ymax = [np.max(data_toplot.loc[m, 'impl_volatility']) for m in mask]

layout = dict(sliders=sliders,
              shapes= [go.layout.Shape(
                        type="line",
                        x0=strike_price_toplot,
                        y0=0.05,
                        x1=strike_price_toplot,
                        y1=0.2,
            line=dict(
                color="Black",
                width=1
            )) for i in range(len(mask))])

fig = dict(data=data_toplot1, layout=layout)

iplot(fig)

In [26]:
date = impl_vol.date == 20170829
days = impl_vol.exdate >0#== 20170915
cp_flag = impl_vol.cp_flag=="C"
volume = impl_vol.volume > 0

toplot = date &\
         days &\
         cp_flag &\
         volume

fig = go.Figure(
        data=go.Scatter(x=impl_vol.loc[toplot, 'strike_price']/1000,
              y=impl_vol.loc[toplot, 'impl_volatility'], mode='markers'))
fig.show()

ValueError: 
    Invalid value of type 'plotly.graph_objs.Scatter' received for the 'data' property of 
        Received value: Scatter({
    'mode': 'markers',
    'x': array([1300., 1800., 1880., ..., 2800., 2900., 3500.]),
    'y': array([     nan,      nan,      nan, ..., 0.107179, 0.102296, 0.119601])
})

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['area', 'bar', 'barpolar', 'box',
                     'candlestick', 'carpet', 'choropleth', 'cone',
                     'contour', 'contourcarpet', 'funnel',
                     'funnelarea', 'heatmap', 'heatmapgl',
                     'histogram', 'histogram2d',
                     'histogram2dcontour', 'isosurface', 'mesh3d',
                     'ohlc', 'parcats', 'p

## Pricing Function

The [Heston process](https://en.wikipedia.org/wiki/Heston_model) is described by the SDE: 

$$ \begin{cases}
dS_t = \mu S_t dt + \sqrt{v_t} S_t dW^1_t \\
dv_t = \kappa (\theta - v_t) dt + \sigma \sqrt{v_t} dW^2_t 
\end{cases}$$

The stock price follows a "geometric Brownian motion" with a stochastic volatility. The square of the volatility (the variance) follows a CIR process.     
Have a look at the notebook **1.2** for more information on the CIR process.

The parameters are:
- $\mu$ drift of the stock process
- $\kappa$ mean reversion coefficient of the variance process
- $\theta$ long term mean of the variance process 
- $\sigma$  volatility coefficient of the variance process
- $\rho$ correlation between $W^1$ and $W^2$ i.e.
$$ dW^1_t dW^2_t = \rho dt $$

We will also require that $2\kappa \theta > \sigma^2$ (Feller condition).




$$
\phi(u,t) = E[exp(iu \operatorname{log}(S_t))|S_0, v_0] \\
\qquad \ = \operatorname{exp}(iu(\operatorname{log}S_0 + (r-q)t)) \\
\qquad \;  \times \operatorname{exp}(\theta\kappa\sigma^{-2}((\kappa - \rho \sigma u i -d)t - 2 \operatorname{log}((1-g \operatorname{e}^{-dt})/(1-g)))) \\
\qquad \; \times \operatorname{exp}(v_0 \sigma^{-2}(\kappa - \rho \sigma u i -d)(1-\operatorname{e}^{-dt})/(1-g\operatorname{e}^{-dt}))
$$

where
$$
d = ((\rho \sigma u i - \kappa)^2 - \theta^2(-iu-u^2))^{\frac{1}{2}} \\
g = (\kappa - \rho \sigma u i - d) / (\kappa - \rho \sigma u i + d)
$$

notes: $\mu=r-q$ in the codes below

In [493]:
def cf_Heston_schoutens(u, S0, t, v0, mu, kappa, theta, rho, sigma):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    """
    xi = kappa - sigma*rho*u*1j
    d  = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g  = (xi-d)/(xi+d)
    
    cf = np.exp((1j*u)*(np.log(S0)+mu*t)) * \
         np.exp((kappa*theta)/(sigma**2) * ( (xi-d)*t - 2*np.log( (1-g*np.exp(-d*t))/(1-g) ))) *\
         np.exp((v0/sigma**2)*(xi-d) * (1-np.exp(-d*t))/(1-g*np.exp(-d*t)) )
    return cf

From paper Carr and Mad
The risk neural probability of finishing in-the-money is:
$$
\mathbb{P}(S_T > K) = \Pi_2 = 1/2 + 1/\pi \int_0^\infty \operatorname{Re}(\frac{\operatorname{e}^{-iu \operatorname{ln}(K)}\phi_T(u)}
{iu}
)du
$$
The delta of the option, dnoted by $\Pi_1$, is numerically obtained as:
$$
\Delta = \Pi_1 = 1/2 + 1/\pi \int_0^\infty \operatorname{Re}(\frac{\operatorname{e}^{-iu \operatorname{ln}(K)}\phi_T(u-i)}
{iu\phi_T(-i)}
)du
$$
Withour dividends and with constant interest rates $r$, the initial option value is:
$$
C = S\Pi_1 - K\operatorname{e}^{-rT}\Pi_2
$$

In [494]:
def Q1(K, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*np.log(K)*1j) / (u*1j)) * 
                                  cf(u=u-1j) / cf(u=-1.0000000000001j) )  
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]

def Q2(K, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*np.log(K)*1j) /(u*1j) * cf(u=u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]


In [495]:
v0 = 0.08                                          # spot variance
rho = -0.8                                         # correlation coefficient
kappa = 3                                          # mean reversion coefficient
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance

mu = 0.05                                          # drift
T = 15                                             # Time to mat
r = 0.05                                           # risk-free rate
K = 100                                            # Stike  
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 

cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens,
                                        S0=S0, t=T, mu=mu, 
                                        v0=v0, theta=theta,
                                        sigma=sigma, kappa=kappa, rho=rho) 

limit_max = 1000      # right limit in the integration                
call = S0 * Q1(K, cf_Heston_schoutens_valuation, limit_max)\
     - K * np.exp(-r*T)* Q2(K, cf_Heston_schoutens_valuation, limit_max)
print(call)

65.27786862732806


In [500]:
r = 0.05                                           # drift
rho = -0.8                                         # correlation coefficient
kappa = 3                                          # mean reversion coefficient
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance
T = 15                                             # Terminal time
K = 100                                            # Stike  
v0 = 0.08                                          # spot variance
S0 = 100                                           # spot stock price 
k = np.log(K/S0)                                   # log moneyness

N = 2000
paths = 2000

dt = T/(N-1)
dt_sq = np.sqrt(dt)
assert(2*kappa*theta > sigma**2)

v_T = np.zeros(paths)   # values of v at T
S_T = np.zeros(paths)   # values of S at T 
v = np.zeros(N)
S = np.zeros(N)

for path in tqdm(range(paths)):
    # Generate random Brownian Motions
    W_S_arr = np.random.normal(loc=0, scale=1, size=N-1 )
    W_v_arr = rho * W_S_arr + np.sqrt(1-rho**2) * np.random.normal(loc=0, scale=1, size=N-1 )
    W_S = W_S_arr
    W_v = W_v_arr
    S[0] = S0           # stock at 0
    v[0] = v0           # variance at 0   

    for t in range(0,N-1):
        v[t+1] = np.fabs( v[t] + kappa*(theta - v[t])*dt + sigma * np.sqrt(v[t]) * dt_sq * W_v[t] )
        S[t+1] = S[t] *  np.exp( (mu - 0.5*v[t])*dt + np.sqrt(v[t]) * dt_sq * W_S[t] )
    S_T[path] = S[N-1]
    v_T[path] = v[N-1]



 74%|███████▍  | 1489/2000 [00:33<00:11, 45.99it/s]


100%|██████████| 2000/2000 [00:44<00:00, 44.84it/s]


In [501]:
DiscountedPayoff = np.exp(-r*T) * np.maximum(S_T-K,0) 
V = scp.mean( DiscountedPayoff )
print(V)

67.23417722278558


In [270]:
DiscountedPayoff = np.exp(-r*T) * np.maximum(S_T-K,0) 
V = scp.mean( DiscountedPayoff )
print(V)

65.4063534722732


In [1]:
import re
import dateutil
import datetime
import xml.etree.ElementTree as et 
# imports to get and handle the XML data
import requests

In [11]:
year = str(2016)
url = '''https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData?$filter=year(NEW_DATE)%20eq%20'''+ year

xml_response = requests.get(url)
xml_text = xml_response.text

etree = et.ElementTree(et.fromstring(xml_text))
root = etree.getroot()
xml_children = root.getchildren()

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if __name__ == '__main__':


In [10]:
print(url)

https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData?$filter=year(NEW_DATE)%20eq%202016


In [352]:
def parseUStreasuryxml(years):
    recs = []
    for y in years:
        year = str(y)
        url = '''https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData?$filter=year(NEW_DATE)%20eq%20'''+ year

        xml_response = requests.get(url)
        xml_text = xml_response.text
        etree = et.ElementTree(et.fromstring(xml_text))
        root = etree.getroot()
        xml_children = root.getchildren()
        for element in xml_children:
            if element.tag is not None:
                # we're looking for records tagged with 'entry'
                if element.tag.endswith('entry'):
                    # indicating we've found a day's yield
                    # get the iterator to step through the elements
                    # looking for the 'content' tag
                    entry_children = element.getiterator()

                    # initialize the date-storage variables
                    curdate = None
                    curdate_dt = None

                    # pull out the content
                    for entry_child in entry_children:
                        # look for the date
                        if entry_child.tag.endswith('NEW_DATE'):
                            # pull out the date
                            curdate = entry_child.text    
                            # and convert to a parsed date value
                            curdate_dt = dateutil.parser.parse(curdate)

                        # otherwise look for an indication of the note term

                        # if this record is associated with a note yield,
                        # pull out the term and the yield
                        elif re.search(r'BC_\d+[a-zA-Z]+$', entry_child.tag):
                            # pull out the date
                            term = re.search('BC_\d+[a-zA-Z]+$', entry_child.tag).group(0)
                            # may have a blank yield
                            # in which case we'll skip
                            if entry_child.text is None:
                                continue
                            else:
                                # otherwise, we can add the item to 
                                rate = float(entry_child.text)
                                recs.append({'term': term,
                                             'date_string': curdate_dt.strftime('%Y-%m-%d'),
                                             'yield': rate,
                                             'dt': curdate_dt})
        # end loop through xml
#         print(url)    
    return pd.DataFrame(recs)

In [372]:
years = range(1996, 2018)
parse_result = parseUStreasuryxml(years)

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



In [375]:
UStreasuryYield = parse_result.pivot(index='date_string', columns='term', values='yield').reset_index()

In [378]:
UStreasuryYield.date_string = UStreasuryYield.date_string.apply(lambda x: x.replace('-',''))

In [381]:
UStreasuryYield = UStreasuryYield.set_index('date_string', drop =False)

In [401]:
UStreasuryYield1 = UStreasuryYield.loc[:,['BC_1MONTH', 'BC_3MONTH', 
                       'BC_6MONTH', 'BC_1YEAR', 'BC_2YEAR']]

In [403]:
parse_result.term.unique()

array(['BC_3MONTH', 'BC_6MONTH', 'BC_1YEAR', 'BC_2YEAR', 'BC_3YEAR',
       'BC_5YEAR', 'BC_7YEAR', 'BC_10YEAR', 'BC_20YEAR', 'BC_30YEAR',
       'BC_30YEARDISPLAY', 'BC_1MONTH'], dtype=object)

In [398]:
impl_vol.loc[:, 't2m'] *12

19960104     2.367123
19960104     2.367123
19960104    17.556164
19960104     2.367123
19960104     2.367123
              ...    
20171229    12.065753
20171229    12.065753
20171229    12.065753
20171229    12.065753
20171229    12.065753
Name: t2m, Length: 12366633, dtype: float64

In [422]:
def rounder2(i, targetList):
    targetList = np.array(targetList)
    idx = np.argmin(np.abs(targetList-i))
    return int(targetList[idx])
    
rounder2(6.1, [1,3,6])

6

In [433]:
impl_vol['riskfree_mo'] = impl_vol.loc[:, 't2m'].apply(lambda x: rounder2(x*12, [1,3,6,12,24]))

In [443]:
cleaned_df = impl_vol.loc[:, ['date', 'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume', 'impl_volatility', 't2m', 'Adj Close', 'riskfree_mo']]

In [444]:
cleaned_df.head()

,date,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,t2m,Adj Close,riskfree_mo
19960104,19960104,C,600000,24.750,25.750,150,0.110927,0.197260,617.700012,3
19960104,19960104,P,615000,10.125,10.875,156,0.125890,0.197260,617.700012,3
19960104,19960104,P,550000,11.625,13.250,5,0.159369,1.463014,617.700012,12
19960104,19960104,P,600000,6.000,6.500,3917,0.136913,0.197260,617.700012,3
19960104,19960104,P,560000,1.750,2.000,3,0.176965,0.197260,617.700012,3


In [468]:
v0 = 0.1109                                          # spot variance
rho = -0.8                                         # correlation coefficient
kappa = 0.1                                          # mean reversion coefficient
theta = 0.10                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance

mu = 0.01                                          # drift
T = 0.197                                             # Time to mat
r = 0.01                                           # risk-free rate
K = 600                                            # Stike  
S0 = 617                                           # spot stock price 

par = (v0, theta, sigma, kappa, rho)

cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens,
                                        S0=S0, t=T, mu=mu, 
                                        v0=v0, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 

limit_max = 1000      # right limit in the integration                
call = S0 * Q1(K, cf_Heston_schoutens_valuation, limit_max)\
     - K * np.exp(-r*T)* Q2(K, cf_Heston_schoutens_valuation, limit_max)
print(call)

45.78031946911659


In [465]:
def call_loss_function(par, **kwargs):
    mu, theta, sigma, kappa, rho = par
    
    S0 = kwargs['S0']
    K =  kwargs['K']
    T =  kwargs['T']
    r =  kwargs['r']
    v0 = kwargs['v0']
    market_price = kwargs['market_price']
    k = np.log(K/S0)
    
    try:
        limit_max = kwargs['limit_max']
    except:
        limit_max = 1000
    
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
    model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
    loss = np.abs((market_price - model_price))
    return loss

In [502]:
v0 = 0.08                                          # spot variance
rho = -0.8                                         # correlation coefficient
kappa = 3                                          # mean reversion coefficient
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance

T = 15                                             # Time to mat
mu = r = 0.05                                      # drift = risk-free rate
K = 100                                            # Stike  
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 

cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens,
                                        S0=S0, t=T, mu=mu, 
                                        v0=v0, theta=theta, 
                                        sigma=sigma, kappa=kappa,
                                        rho=rho) 

limit_max = 1000      # right limit in the integration                
call = S0 * Q1(K, cf_Heston_schoutens_valuation, limit_max)\
     - K * np.exp(-r*T)* Q2(K, cf_Heston_schoutens_valuation, limit_max)
print(call)

65.27786862732806


In [522]:
def loss_function(par2opt, market, cf_Heston_schoutens):
    v0 = par2opt[0]
    theta = par2opt[1]
    sigma = par2opt[2]
    kappa = par2opt[3]
    rho = par2opt[4]
    
    S0 = market[0]
    T = market[1]
    mu = market[2]
    market_price = market[3]
    
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens,
                                        S0=S0, t=T, mu=mu, 
                                        v0=v0, theta=theta, 
                                        sigma=sigma, kappa=kappa,
                                        rho=rho) 

    limit_max = 1000      # right limit in the integration                
    call = S0 * Q1(K, cf_Heston_schoutens_valuation, limit_max)\
     - K * np.exp(-r*T)* Q2(K, cf_Heston_schoutens_valuation, limit_max)
    print(call)
    return np.abs(call - market_price)

par = [v0, theta, sigma, kappa, rho]    
market = [S0, T, mu, 65]
loss_function(par, market, cf_Heston_schoutens)

65.27786862732806


In [523]:
loss_function2opt = partial(loss_function, 
                            cf_Heston_schoutens = cf_Heston_schoutens, 
                            market = market)

In [524]:
loss_function2opt(par)

65.27786862732806


In [525]:
init_par = np.random.uniform(size=5)
res = minimize(loss_function2opt, init_par, method='nelder-mead',
               options={'xtol': 1e-8, 'disp': True})

92.36939244249977
92.39804421403429
92.94710139167569
92.21792195089157
92.55206448292354
92.40826980012423
91.78565568565132
91.12334377811622
91.65244414593825
91.51192429580733
91.13924758775924
90.62986250593187
89.6204230079323
89.67079403824025
89.49565581999312
88.16594522568936
88.2130497682448
87.31724548631983
84.70084307705059
84.38230670962918
79.24466794860274
82.13176253132063
77.29489040289013
64.50658718912055
69.60156195278788
56.00772014632898
2.8660602518500204e+61
79.41204587899614
53.83371768190514
-7.1330265724447625e+180
73.57999672776312
nan


/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning:

overflow encountered in exp

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in cdouble_scalars

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning:

overflow encountered in cdouble_scalars

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in cdouble_scalars

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in cdouble_scalars

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

/Users/francis/opt/anaconda3/lib/python3.7/

72.93879474653001
78.14272082364863
60.89243070927975
78.16048112011659
62.5651312630337
57.62917107686969
1.9512451464721545e+46
68.51222216254806
71.92735418169192
68.83107648675089
60.20047086345508
67.47795245213985
71.32910395826639
63.75363931724248
61.71665963423292
58.988029324630475
66.36639439332151
67.9939910105864
66.51662286419933
61.84165101894676
66.15175805444099
68.26100524877079
64.0352090626814
63.37928890609892
65.75748414368985
63.250862735135016
65.62108706498213
66.66324962395115
64.49358865258316
63.57334191426165
65.52808335336348
66.32932256683215
64.613491706206
64.13699305802784
65.36093724524042
64.1647962363569
65.26610666346821
64.15677593613712
65.19216953071943
65.48963565148557
65.82075873481546
64.85103408600116
64.62840518574802
65.50600642386564
64.83738011498156
65.57886481208232
64.86534084005501
64.64116697598936
64.82276590636303
64.55701091500521
65.09084495967524
64.59222051990344
65.04339981162445
65.05278097293098
65.12515363797408
65.214246

In [527]:
res.x

array([0.31018429, 0.10927156, 1.21781898, 0.57602815, 0.12701734])

In [533]:
parse_result.date_string = parse_result.date_string.apply(lambda x: x.replace("-",""))

In [535]:
parse_result.term.unique()

array(['BC_3MONTH', 'BC_6MONTH', 'BC_1YEAR', 'BC_2YEAR', 'BC_3YEAR',
       'BC_5YEAR', 'BC_7YEAR', 'BC_10YEAR', 'BC_20YEAR', 'BC_30YEAR',
       'BC_30YEARDISPLAY', 'BC_1MONTH'], dtype=object)

In [544]:
terms = parse_result.term.unique()
months = [3,6,12,24,36,60,84,120,240,360,'360_display', 1]
parse_result['months'] = 0

for i, term in enumerate(terms):
    idx = parse_result.term == term
    parse_result['months'].loc[idx] = months[i]
    

# usRf = parse_result.loc

In [555]:
parse_result.groupby(['term']).max()

,date_string,yield,dt,months
term,,,,
BC_10YEAR,20171229,7.06,2017-12-29,120
BC_1MONTH,20171229,5.27,2017-12-29,1
BC_1YEAR,20171229,6.44,2017-12-29,12
BC_20YEAR,20171229,7.35,2017-12-29,240
BC_2YEAR,20171229,6.93,2017-12-29,24
BC_30YEAR,20171229,7.19,2017-12-29,360
BC_30YEARDISPLAY,20171229,4.76,2017-12-29,360_display
BC_3MONTH,20171229,6.42,2017-12-29,3
BC_3YEAR,20171229,6.88,2017-12-29,36


In [663]:
a = parse_result.rename(columns={'date_string':'date', 'months':'riskfree_mo'})
# a = a.set_index(['date', 'riskfree_mo'])
a.date = a.date.apply(str)
a.riskfree_mo = a.riskfree_mo.apply(str)


In [637]:
cleaned_df.date = cleaned_df.date.apply(str)
cleaned_df.riskfree_mo = cleaned_df.riskfree_mo.apply(str)

In [600]:
cleaned_df.riskfree_mo

19960104     3
19960104     3
19960104    12
19960104     3
19960104     3
            ..
20171229    12
20171229    12
20171229    12
20171229    12
20171229    12
Name: riskfree_mo, Length: 12366633, dtype: object

In [577]:
a.head()

term  yield         dt
date     riskfree_mo                             
19961231 3            BC_3MONTH   5.21 1996-12-31
         6            BC_6MONTH   5.33 1996-12-31
         12            BC_1YEAR   5.51 1996-12-31
         24            BC_2YEAR   5.88 1996-12-31
         36            BC_3YEAR   6.04 1996-12-31

In [605]:
cleaned_df.head()

,date,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,t2m,Adj Close,riskfree_mo
19960104,19960104,C,600000,24.750,25.750,150,0.110927,0.197260,617.700012,3
19960104,19960104,P,615000,10.125,10.875,156,0.125890,0.197260,617.700012,3
19960104,19960104,P,550000,11.625,13.250,5,0.159369,1.463014,617.700012,12
19960104,19960104,P,600000,6.000,6.500,3917,0.136913,0.197260,617.700012,3
19960104,19960104,P,560000,1.750,2.000,3,0.176965,0.197260,617.700012,3


In [664]:
c = cleaned_df.merge(a, how='left',
                     left_on=['date', 'riskfree_mo'], 
                     right_on=['date', 'riskfree_mo'],)

In [27]:
import pickle 
# pickle_out = open("./data/cleaned_data","wb")
# pickle.dump(c, pickle_out)
# pickle_out.close()

pickle_in = open("./data/cleaned_data","rb")
example_dict = pickle.load(pickle_in)
pickle_in.close()


In [28]:
example_dict

,date,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,t2m,Adj Close,riskfree_mo,term,yield,dt
0,19960104,C,600000,24.750,25.750,150,0.110927,0.197260,617.700012,3,BC_3MONTH,5.19,1996-01-04
1,19960104,P,615000,10.125,10.875,156,0.125890,0.197260,617.700012,3,BC_3MONTH,5.19,1996-01-04
2,19960104,P,550000,11.625,13.250,5,0.159369,1.463014,617.700012,12,BC_1YEAR,5.19,1996-01-04
3,19960104,P,600000,6.000,6.500,3917,0.136913,0.197260,617.700012,3,BC_3MONTH,5.19,1996-01-04
4,19960104,P,560000,1.750,2.000,3,0.176965,0.197260,617.700012,3,BC_3MONTH,5.19,1996-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12366628,20171229,P,2950000,282.300,285.000,0,0.099606,1.005479,NaN,12,BC_1YEAR,1.76,2017-12-29
12366629,20171229,P,3000000,321.800,324.700,0,0.095434,1.005479,NaN,12,BC_1YEAR,1.76,2017-12-29
12366630,20171229,P,3050000,364.700,367.600,0,0.092612,1.005479,NaN,12,BC_1YEAR,1.76,2017-12-29
12366631,20171229,P,3100000,399.300,419.600,0,0.084638,1.005479,NaN,12,BC_1YEAR,1.76,2017-12-29
